In [1]:
import tensorflow as tf
import json
from trainer import Trainer
from utils import map_word_to_vector
import numpy as np
import os

In [2]:
with open('102/checkpoints/model_configs.json', 'r') as inp:
    model_configs = json.load(inp)

In [3]:
with open('SentimentAnalysis//word2idx.json', 'r') as inp:
    new_word2idx = json.load(inp)
with open('SentimentAnalysis/char2idx.json', 'r') as inp:
    new_char2idx = json.load(inp)
with open('102/word2idx.json', 'r') as inp:
    old_word2idx = json.load(inp)
with open('102/char2idx.json', 'r') as inp:
    old_char2idx = json.load(inp)

In [4]:
VERSION = 102
params = dict(
    model_configs = model_configs,
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 8192,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

In [5]:
trainer = Trainer(**params)

In [6]:
trainer.build()

INFO:tensorflow:Restoring parameters from 102/checkpoints/train/model.cpkt-1046000


Restoring parameters from 102/checkpoints/train/model.cpkt-1046000


In [7]:
model_configs['vocab_size'] = len(new_word2idx) + 1
model_configs['char_vocab_size'] = len(new_char2idx) + 1

In [8]:
tf.global_variables()

[<tf.Variable 'LanguageModel/decode_W:0' shape=(512, 153360) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/decode_b:0' shape=(153360,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/embedding_weight:0' shape=(1005, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/kernel:0' shape=(1, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/kernel:0' shape=(2, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/kernel:0' shape=(3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/kernel:0' shape=(4, 16, 64) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/bias:0' s

In [9]:
with tf.variable_scope('LanguageModel', reuse=True):
    W_var = tf.get_variable('decode_W')
    b_var = tf.get_variable('decode_b')
    W_ema_var = trainer.ema.average(W_var)
    b_ema_var = trainer.ema.average(b_var)

In [10]:
W_val = trainer.session.run(W_var)
b_val = trainer.session.run(b_var)
W_ema_val = trainer.session.run(W_ema_var)
b_ema_val = trainer.session.run(b_ema_var)

In [11]:
new_W_val = map_word_to_vector(new_word2idx, old_word2idx, W_val.T).T
new_b_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_val, axis=-1)), axis=-1)
new_W_ema_val = map_word_to_vector(new_word2idx, old_word2idx, W_ema_val.T).T
new_b_ema_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_ema_val, axis=-1)), axis=-1)

In [12]:
assign_W = tf.assign(W_var, new_W_val, validate_shape=False)
assign_b = tf.assign(b_var, new_b_val, validate_shape=False)
assign_W_ema = tf.assign(W_ema_var, new_W_ema_val, validate_shape=False)
assign_b_ema = tf.assign(b_ema_var, new_b_ema_val, validate_shape=False)

In [13]:
trainer.session.run([assign_W, assign_b, assign_W_ema, assign_b_ema])

[array([[-0.02900317,  0.00242317,  0.02402635, ...,  0.0096975 ,
          0.00533582, -0.01156895],
        [ 0.0299977 , -0.10840327, -0.11387965, ...,  0.00502902,
         -0.01067196, -0.03624642],
        [ 0.00626211, -0.02825956,  0.17951813, ...,  0.02727997,
         -0.03085063, -0.02527289],
        ...,
        [ 0.00189254, -0.05637207, -0.12711304, ...,  0.01247254,
         -0.00112364, -0.00282849],
        [-0.04899433,  0.02262462, -0.06198847, ...,  0.00792436,
         -0.02380779, -0.00877183],
        [-0.00833268,  0.11103068,  0.02551415, ..., -0.00747284,
         -0.0207533 , -0.00675184]], dtype=float32),
 array([ 0.01684981,  8.610944  ,  4.7202516 , ...,  0.06646784,
         0.0553963 , -0.04711184], dtype=float32),
 array([[ 0.02247547, -0.00666038,  0.04065513, ...,  0.03241943,
         -0.00203702,  0.01187309],
        [ 0.00830166, -0.07638112, -0.1119715 , ...,  0.03054176,
          0.02697206, -0.02376601],
        [-0.01270556, -0.01647925,  0.

In [15]:
trainer.checkpoint_dir = '102_sa/checkpoints/'
trainer.model_configs = model_configs
trainer.save_configs()

In [16]:
trainer.train_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'train', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'102_sa/checkpoints/train/model.cpkt-1046000'

In [17]:
trainer.test_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'test', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'102_sa/checkpoints/test/model.cpkt-1046000'